In [5]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [6]:
from minio import Minio

minio_client = Minio(
	endpoint=os.getenv('S3_ENDPOINT'),
	access_key=os.getenv('ACCESS_KEY_ID'),
	secret_key=os.getenv('SECRET_ACCESS_KEY'),
	secure=True,
)

In [34]:
model_id = '66c32647250225d1d36e735e'

# minio_client.fget_object(os.getenv('BUCKET'), f"{model_id}/catalog.v1.parquet.backup", f"kk/catalog.v1.parquet")
minio_client.fget_object(os.getenv('BUCKET'), f"{model_id}/catalog.v1.parquet", f"kk/catalog.v1.parquet")

In [35]:
import geopandas as gpd

df = gpd.read_parquet("kk/catalog.v1.parquet")

df

,type,stac_version,stac_extensions,datetime,id,bbox,geometry,assets,links,repository
0,Feature,1.0.0,[],2025-03-25 13:50:39.232448,model.onnx,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '99581a42cd56f66b1649f5...,[],eotdl


In [36]:
properties={ 
	"mlm:name": "model.onnx", # name of the asset ? otherwise, how can we know which asset to use ?
	"mlm:framework": "ONNX",  # only framework support for now
	"mlm:architecture": "U-Net",
	"mlm:tasks": ["segmentation"], # https://github.com/crim-ca/mlm-extension?tab=readme-ov-file#task-enum
	"mlm:input": { # https://github.com/crim-ca/mlm-extension?tab=readme-ov-file#model-input-object
		"name": "RGB statellite image (HR)",
		"bands": [
			"red",
			"green",
			"blue"
		],
		"input": { # https://github.com/crim-ca/mlm-extension?tab=readme-ov-file#input-structure-object
			"shape": [
				-1,
				3,
				-1, # should be divisble by 16
				-1 # should be divisble by 16
			],
			"dim_order": [
				"batch",
				"channel",
				"height",
				"width"
			],
			"data_type": "float32",
			# we should add here the resize to nearest divisible by 16
			# "pre_processing_function": { # https://github.com/crim-ca/mlm-extension?tab=readme-ov-file#processing-expression
			# 	"format": 
			# 	"expression": 
			# }
			"description": "Model trained with 1024x1024 RGB HR images, but can work with other dimensions as long as they are divisible by 16"
		}
	},
	"mlm:output": {
		"name": "road binary mask",
		"tasks": ["segmentation"], # redundant ?
		"result": { # https://github.com/crim-ca/mlm-extension?tab=readme-ov-file#result-structure-object
			"shape": [-1, 1, -1, -1],
			"dim_order": [
				"batch",
				"channel",
				"height",
				"width"
			],
			"data_type": "uint8",
			"description": "Binary mask of the road segmentation. 1 for road, 0 for background",
			# "post_processing_function": { # https://github.com/crim-ca/mlm-extension?tab=readme-ov-file#processing-expression
			# }
		},
	},
}
    
stac_extensions=['https://crim-ca.github.io/mlm-extension/v1.2.0/schema.json']

In [37]:
# Update the DataFrame in a single operation
df.loc[0, 'stac_extensions'] = stac_extensions
df.loc[0, 'mlm:name'] = properties['mlm:name']
df.loc[0, 'mlm:framework'] = properties['mlm:framework']
df.loc[0, 'mlm:architecture'] = properties['mlm:architecture']
df.loc[0, 'mlm:tasks'] = properties['mlm:tasks']  
df['mlm:input'] = [properties['mlm:input']]
df['mlm:output'] = [properties['mlm:output']]

In [38]:
df

,type,stac_version,stac_extensions,datetime,id,bbox,geometry,assets,links,repository,mlm:name,mlm:framework,mlm:architecture,mlm:tasks,mlm:input,mlm:output
0,Feature,1.0.0,[https://crim-ca.github.io/mlm-extension/v1.2....,2025-03-25 13:50:39.232448,model.onnx,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '99581a42cd56f66b1649f5...,[],eotdl,model.onnx,ONNX,U-Net,[segmentation],"{'name': 'RGB statellite image (HR)', 'bands':...","{'name': 'road binary mask', 'tasks': ['segmen..."


In [40]:
# minio_client.fput_object(os.getenv('BUCKET'), f"{model_id}/catalog.v1.parquet.backup", f"kk/catalog.v1.parquet")

df.to_parquet("kk/catalog.v1.parquet")

In [41]:
df = gpd.read_parquet("kk/catalog.v1.parquet")

df

,type,stac_version,stac_extensions,datetime,id,bbox,geometry,assets,links,repository,mlm:name,mlm:framework,mlm:architecture,mlm:tasks,mlm:input,mlm:output
0,Feature,1.0.0,[https://crim-ca.github.io/mlm-extension/v1.2....,2025-03-25 13:50:39.232448,model.onnx,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '99581a42cd56f66b1649f5...,[],eotdl,model.onnx,ONNX,U-Net,[segmentation],"{'bands': ['red', 'green', 'blue'], 'input': {...","{'name': 'road binary mask', 'result': {'data_..."


In [42]:
minio_client.fput_object(os.getenv('BUCKET'), f"{model_id}/catalog.v1.parquet", f"kk/catalog.v1.parquet")
